 ## **Exercise 2.04 - Guided Exercise**

### Fitting a Log Linear Regression Model Using the Statsmodels formula API

In [ ]:
#importing modules
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

%matplotlib inline
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import pandas as pd
import numpy as np
import patsy
from statsmodels.graphics.correlation import plot_corr
from sklearn.model_selection import train_test_split
plt.style.use('seaborn')



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


LOADING MIAMI_HOUSING DATASET AND DISPLAYING FIRST 5 ENTRIES

In [ ]:

miami_data = pd.read_csv\
('https://raw.githubusercontent.com/fenago/DSBook/main/miami-housing.csv')
miami_data.head()

Check for missing values (null values) in the DataFram

In [ ]:
raw_data = miami_data.dropna()


In [ ]:
#Checking for duplicate data and removing it 
raw_data = raw_data.drop_duplicates()

In [ ]:
renamedMiamiData = raw_data.rename\
(columns = {\
'LATITUDE':'LATITUDE',\
' LONGITUDE ':'lONGITUDE',\
'PARCELNO ':'Parcel Number',\
'SALE_PRC':'SalePrice',\
'LND_SQFOOT':'LandLength'})
renamedMiamiData.head()

Divide the DataFrame into training and test sets

In [ ]:
X = renamedMiamiData.drop('SalePrice', axis = 1)
y = renamedMiamiData[['SalePrice']]
seed = 10
test_data_size = 0.3
X_train, X_test, \
y_train, y_test = train_test_split(X, y, \
test_size = test_data_size, \
random_state = seed)
train_data = pd.concat([X_train, y_train], axis = 1)
test_data = pd.concat([X_test, y_test], axis = 1)

 Defining a Log - linear regression model and storing it in a variable

In [ ]:
#----Write your code here----#



#-------Solution:---------#
logLinear_model = smf.ols\
              (formula='np.log(SalePrice) ~ LandLength',\
              data=train_data)

Calling the .fit method of the model instance and assigning the results of the method to a linearModelResult variable,

In [ ]:
FinalLogLinearModel = logLinear_model.fit()

Printing  the summary stored in Model -  Regression analysis results

In [ ]:
print(FinalLogLinearModel.summary())